In [ ]:
import paho.mqtt.client as mqtt
import json

In [ ]:
def save_json(filename, data):
    with open(f'data/{filename}.json', "w") as f:
        json.dump(data, f, indent=4)

def read_json(filename):
    with open(f'data/{filename}.json', 'r') as file:
        data = json.load(file)
    return data

In [ ]:
keys = read_json('broker')

broker = keys['broker']
port = keys['port']
topic = keys['topics']

In [ ]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Conectado al broker MQTT")
        client.subscribe(topic)
    else:
        print("Error de conexión, código:", rc)

In [ ]:
def save_data():
    part_0 = read_json('part_0')
    part_1 = read_json('part_1')
    
    total_part = [{**part_0,**part_1}]
    
    # save data in history
    data = read_json('history')
    data.extend(total_part)
    save_json('history',data)
    
    # save data un last_update
    save_json('last_update',total_part)
    
    print('saving data... 100%')

In [ ]:
def on_message(client, userdata, message):
    global dicc_total, temp
    
    temp = json.loads(str(message.payload.decode()).replace("\'}","").replace("{\'" ,""))
    
    if 1 in temp['data'][0]['point'][1:][0].values():
        part_0 = temp['data'][0]['point'][1:]
        dicc = {item['id']:item['val'] for item in part_0}
        save_json('part_0',dicc)
    else:
        part_1 = temp['data'][0]['point'][1:]
        dicc = {item['id']:item['val'] for item in part_1}        
        save_json('part_1',dicc)
        save_data()

In [ ]:
# Crea el cliente MQTT
client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

client.connect(broker, port)

# Mantén el cliente en ejecución para recibir mensajes
client.loop_forever()